<a href="https://colab.research.google.com/github/slitvinov/CUP3D/blob/AMR/fish.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/slitvinov/CUP3D/blob/AMR/fish.ipynb)

In [1]:
%%sh
git clone https://github.com/slitvinov/CUP3D.git

Cloning into 'CUP3D'...


In [ ]:
%%sh
sudo apt install libhdf5-mpich-dev libgsl-dev

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  hwloc-nox libgsl27 libgslcblas0 libhdf5-mpich-103-1 libhdf5-mpich-cpp-103-1
  libhdf5-mpich-fortran-102 libhdf5-mpich-hl-100 libhdf5-mpich-hl-cpp-100
  libhdf5-mpich-hl-fortran-100 libmpich-dev libmpich12 libslurm37 mpich
Suggested packages:
  gsl-ref-psdoc | gsl-doc-pdf | gsl-doc-info | gsl-ref-html libhdf5-doc
  mpich-doc
The following NEW packages will be installed:
  hwloc-nox libgsl-dev libgsl27 libgslcblas0 libhdf5-mpich-103-1
  libhdf5-mpich-cpp-103-1 libhdf5-mpich-dev libhdf5-mpich-fortran-102
  libhdf5-mpich-hl-100 libhdf5-mpich-hl-cpp-100 libhdf5-mpich-hl-fortran-100
  libmpich-dev libmpich12 libslurm37 mpich
0 upgraded, 15 newly installed, 0 to remove and 45 not upgraded.
Need to get 21.0 MB of archives.
After this operation, 131 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libgslcbla



debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 15.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


In [ ]:
%%sh
cd CUP3D
MPICC=h5c++.mpich sh build.sh

In file included from Cubism/Cubism/BlockLab.h:4,
                 from source/Obstacles/../Definitions.h:19,
                 from source/Obstacles/../ObstacleBlock.h:12,
                 from source/Obstacles/Obstacle.h:9,
                 from source/Obstacles/Fish.h:9,
                 from source/Obstacles/Naca.h:9,
                 from source/Obstacles/SmartNaca.h:9,
                 from source/Obstacles/SmartNaca.cpp:7:
Cubism/Cubism/Matrix3D.h: In member function ‘void cubism::Matrix3D<DataType, allocator>::_Setup(unsigned int, unsigned int, unsigned int) [with DataType = cubism::ScalarElement<double>; allocator = cubismup3d::aligned_block_allocator]’:
Cubism/Cubism/Matrix3D.h:57:21: warning: ignoring return value of ‘int posix_memalign(void**, size_t, size_t)’ declared with attribute ‘warn_unused_result’ [-Wunused-result]
   57 |       posix_memalign((void **)&m_pData, std::max(8, CUBISM_ALIGNMENT), sizeof(DataType) * m_nElements);
      |       ~~~~~~~~~~~~~~^~~~~~~~~~~~~~~

In [ ]:
%%sh
cd CUP3D
OMPI_ALLOW_RUN_AS_ROOT=1 OMPI_ALLOW_RUN_AS_ROOT_CONFIRM=1 bash run.sh

Valid =  1
Valid =  2
Cubism UP 3D (velocity-pressure 3D incompressible Navier-Stokes solver)
Running in RELEASE mode!
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
* Summary:
*    Parameter read from command line:                20
*    Parameter read from   0 file(s):                 0
*    Parameter read from defaults in code:            0
*    Total number of parameter read from all sources: 20
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
* Command Line:
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
CFL...............................................: 0.4
Ctol..............................................: 0.1
Rtol..............................................: 5.0
bMeanConstraint...................................: 2
bpdx..............................................: 8
bpdy..............................................: 4
bpdz..............................................: 4
dumpOmegaX.....................

--------------------------------------------------------------------------
Primary job  terminated normally, but 1 process returned
a non-zero exit code. Per user-direction, the job has been aborted.
--------------------------------------------------------------------------
--------------------------------------------------------------------------
mpiexec noticed that process rank 0 with PID 0 on node cdb6c3645348 exited on signal 9 (Killed).
--------------------------------------------------------------------------


CalledProcessError: Command 'b'cd CUP3D\nOMPI_ALLOW_RUN_AS_ROOT=1 OMPI_ALLOW_RUN_AS_ROOT_CONFIRM=1 bash run.sh\n'' returned non-zero exit status 137.